In [27]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import time

spark = SparkSession.builder \
    .appName("SQLite with PySpark") \
    .config("spark.jars", "C:\\Users\\dongh\\Documents\\GitHub\\Flask-Banking-Application\\instance\\sqlite-jdbc-3.43.0.0.jar") \
    .getOrCreate()

In [12]:
# Đường dẫn tới file SQLite
db_path = "instance/bank_app.db"
table_name = "Application"

def read_sqlite_data(db_path, table_name):
# Đọc dữ liệu từ SQLite
    df = spark.read.format("jdbc") \
        .option("url", f"jdbc:sqlite:{db_path}") \
        .option("dbtable", table_name) \
        .option("driver", "org.sqlite.JDBC") \
        .load()
    return df


In [23]:
application_df = read_sqlite_data(db_path, table_name)
application_df.show()

+---+-------------+---+------+--------+-------+------+--------+----------+
| id|         name|age|employ|creddebt|debtinc|income| othdebt|    result|
+---+-------------+---+------+--------+-------+------+--------+----------+
|  1|        Đônng| 13|   4.0|     1.0|    1.0|   1.0|5.008608|No Default|
|  2|            1| 13|   4.0|     1.0|    1.0|   1.0|5.008608|No Default|
|  3|        Dongg| 19|   4.0|     1.0|    1.0|   1.0|5.008608|No Default|
|  4|Đônng Nguuyễn| 13|   1.0|     1.0|    1.0|  14.0|    14.0|No Default|
|  5|         Dông| 10|   1.0|     1.0|    1.0|  14.0|    14.0|No Default|
|  6|         Dông| 10|   1.0|     1.0|    1.0|  14.0|    14.0|No Default|
|  7|        Đônng| 41|  17.0|   11.36|    9.3| 176.0|     5.0|No Default|
|  8|        Đônng| 41|  17.0|  11.359|    9.3| 176.0|5.008608|No Default|
|  9|        Đônng| 41|   1.0|     2.0|    9.3| 176.0|5.008608|No Default|
| 10|        Đônng| 41|   1.0|     2.0|    9.3| 176.0|5.008608|No Default|
| 11|        Đônng| 41|  

In [16]:
application_df.createOrReplaceTempView("Application")
default_rate = spark.sql("""
    SELECT result, COUNT(*) AS count, 
           ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS percentage
    FROM Application
    GROUP BY result
""")
default_rate.show()


+----------+-----+----------+
|    result|count|percentage|
+----------+-----+----------+
|   Default|    5|     31.25|
|No Default|   11|     68.75|
+----------+-----+----------+



In [18]:
age_group = spark.sql("""
    SELECT 
        CASE 
            WHEN age <= 20 THEN '0-20'
            WHEN age <= 40 THEN '21-40'
            WHEN age <= 60 THEN '41-60'
            WHEN age <= 80 THEN '61-80'
            ELSE '81+' 
        END AS age_group,
        COUNT(*) AS count
    FROM Application
    GROUP BY age_group
    ORDER BY age_group
""")
age_group.show()


+---------+-----+
|age_group|count|
+---------+-----+
|     0-20|    7|
|    21-40|    1|
|    41-60|    8|
+---------+-----+



In [19]:
total_applications = spark.sql("SELECT COUNT(*) AS total FROM Application")
total_applications.show()


+-----+
|total|
+-----+
|   16|
+-----+



In [39]:
def query_with_count_in_buckets(application_df):
    
    # Đăng ký DataFrame như một view tạm thời để truy vấn bằng SQL
    application_df.createOrReplaceTempView("Application")
    
    # Truy vấn SQL với phân chia các khoảng income và debtinc
    query = """
    SELECT 
        CASE 
            WHEN income <= 20 THEN '0-20'
            WHEN income <= 40 THEN '21-40'
            WHEN income <= 60 THEN '41-60'
            WHEN income <= 80 THEN '61-80'
            ELSE '81+' 
        END AS income_group,
        COUNT(*) AS count
    FROM Application
    GROUP BY income_group
    ORDER BY income_group
    """
    
    # Thực thi câu truy vấn SQL
    result_df = spark.sql(query)
    
    # Hiển thị kết quả
    result_df.show()

In [40]:
data = query_with_count_in_buckets(application_df)

+------------+-----+
|income_group|count|
+------------+-----+
|        0-20|    6|
|         81+|   10|
+------------+-----+

